In [1]:
import numpy as np
from docx import Document
from docx.shared import Pt

In [2]:
f='gstr3b'
fname=f'{f}.docx'

doc=Document(fname)
all_tables=[]
for table in doc.tables:
    rows=len(table.rows)
    cols=len(table.columns)
    data=np.empty((rows,cols), dtype=object)
    for i,row in enumerate(table.rows):
        for j,cell in enumerate(row.cells):
            data[i,j]=cell.text.strip()
    all_tables.append(data)

In [3]:
l=['3.1 Details of Outward supplies and inward supplies liable to reverse charge (other than those covered by Table 3.1.1)',
'3.1.1 Details of Supplies notified under section 9(5) of the CGST Act, 2017 and corresponding provisions in IGST/UTGST/ SGST Acts',
'3.2 Out of supplies made in 3.1 (a) and 3.1.1 (i), details of inter-state supplies made',
'4. Eligible ITC',
'4. Eligible ITC',
'5 Values of exempt, nil-rated and non-GST inward supplies',
'5.1 Interest and Late fee for previous tax period',
'6.1 Payment of tax',
'Breakup of tax liability declared (for interest computation)']	

In [4]:
all_tables[6]=np.vstack(([all_tables[5][0]], all_tables[6]))
for k in range(len(l)):
    x=all_tables[k+2]
    y=np.array([l[k]]*(x.shape[1]))
    all_tables[k+2]=np.vstack((y, x))

In [5]:
lists=[]
for i in range(2):
    for row_index, row in enumerate(all_tables[i]):
        for col_index, value in enumerate(row):
            if len(value)==0:
                d={}
                d['Description']=''
                d['Row']=all_tables[i][row_index][-2]
                d['Value']='NaN'
                d['Column']=''
                d['Coordinates']=(row_index,col_index,i)
                lists.append(d)

In [6]:
for table_index in range(2,len(all_tables)):
    x=all_tables[table_index]
    for row_index, row in enumerate(x):
        for col_index, value in enumerate(row):
            if value=='0.00':
                d={}
                d['Description']=x[0,0]
                # d['Row']=f"{x[1,0]} | {x[row_index,0]}"
                d['Row']=x[row_index,0]
                d['Column']=x[1,col_index]
                d['Value']='NaN'
                d['Coordinates']=(row_index,col_index,table_index)
                lists.append(d)
lists

[{'Description': '',
  'Row': 'Year',
  'Value': 'NaN',
  'Column': '',
  'Coordinates': (0, 1, 0)},
 {'Description': '',
  'Row': 'Period',
  'Value': 'NaN',
  'Column': '',
  'Coordinates': (1, 1, 0)},
 {'Description': '',
  'Row': '1. GSTIN',
  'Value': 'NaN',
  'Column': '',
  'Coordinates': (0, 1, 1)},
 {'Description': '',
  'Row': '2(a). Legal name of the registered person',
  'Value': 'NaN',
  'Column': '',
  'Coordinates': (1, 1, 1)},
 {'Description': '',
  'Row': '2(b). Trade name, if any',
  'Value': 'NaN',
  'Column': '',
  'Coordinates': (2, 1, 1)},
 {'Description': '',
  'Row': '2(c). ARN',
  'Value': 'NaN',
  'Column': '',
  'Coordinates': (3, 1, 1)},
 {'Description': '',
  'Row': '2(d). Date of ARN',
  'Value': 'NaN',
  'Column': '',
  'Coordinates': (4, 1, 1)},
 {'Description': '3.1 Details of Outward supplies and inward supplies liable to reverse charge (other than those covered by Table 3.1.1)',
  'Row': '(a) Outward taxable supplies (other than zero rated, nil rated 

In [7]:
o={}
o['Total taxable value']='Value'
o['Integrated tax']='IGST'
o['Central tax']='CGST'
o['State/UT tax']='SGST'
o['State/ UT tax']='SGST'
o['Cess']='Cess'
o['']=''

k=[f]
for i_index, i in enumerate(lists):
    if i['Column'] in o.keys() and i['Row'] in o.keys():
        s=' | '.join([f"{i_index}.",o[i['Row']],o[i['Column']],i['Description']])
    elif i['Row'] in o.keys():
        s=' | '.join([f"{i_index}.",o[i['Row']],i['Column'],i['Description']])
    elif i['Column'] in o.keys():
        s=' | '.join([f"{i_index}.",i['Row'],o[i['Column']],i['Description']])
    else:
        s=' | '.join([f"{i_index}.",i['Row'],i['Column'],i['Description']])
    k.append(s)
k

['gstr3b',
 '0. | Year |  | ',
 '1. | Period |  | ',
 '2. | 1. GSTIN |  | ',
 '3. | 2(a). Legal name of the registered person |  | ',
 '4. | 2(b). Trade name, if any |  | ',
 '5. | 2(c). ARN |  | ',
 '6. | 2(d). Date of ARN |  | ',
 '7. | (a) Outward taxable supplies (other than zero rated, nil rated and exempted) | Value | 3.1 Details of Outward supplies and inward supplies liable to reverse charge (other than those covered by Table 3.1.1)',
 '8. | (a) Outward taxable supplies (other than zero rated, nil rated and exempted) | IGST | 3.1 Details of Outward supplies and inward supplies liable to reverse charge (other than those covered by Table 3.1.1)',
 '9. | (a) Outward taxable supplies (other than zero rated, nil rated and exempted) | CGST | 3.1 Details of Outward supplies and inward supplies liable to reverse charge (other than those covered by Table 3.1.1)',
 '10. | (a) Outward taxable supplies (other than zero rated, nil rated and exempted) | SGST | 3.1 Details of Outward supplies

In [20]:
import pickle
with open('gstr3b_params.pkl', 'wb') as f:
    pickle.dump(k, f)